In [1]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

from pyspark.sql import Row

In [21]:
contest_sol_end_customer = sqlContext.sql('select * from edso_ignite.contest_savm').repartition(100).cache()

In [33]:
contest_sol_end_customer = contest_sol_end_customer.select([c for c in contest_sol_end_customer.columns if c != 'province'])

In [31]:
contest_sol_end_customer.take(5)

[Row(party_id=1052727.0, parent_party_id=42499.0, party_name=u'PIZZA HUT, INC.', node_type=u'BR', address1=u'616 N 15TH ST', address2=None, address3=None, address4=None, city=u'FORT DODGE', county=u'WEBSTER', state=u'IA', province=None, postal_code=u'50501', postal_code_extn=u'3322', country_code=u'US', street_name=u'15TH', street_number=u'616', street_direction=u'N', street_type=u'ST', geo_valid_status=u'GEO_VALID', completenes_status=u'COMPLETE', cleansed_status=u'CLEANSED', start_date=u'2002-11-12 04:57:20.0', end_date=u'4712-12-31 00:00:00.0', program_id=None, request_id=None, created_by=1116.0, last_updated_by=332355.0, creation_date=u'2002-11-12 04:57:20.0', last_update_date=u'2014-11-16 13:31:33.0', certified_date=u'2009-04-13 03:54:19.0', site_expl_id=479696471.0, conflict_batch_id=87615798.0, sa_member_id=3755080.0, parent_sa_member_id=-1.0, party_level=2.0, link_party_id=42499.0, link_party_type=u'ORGANIZATION', split_pct=50.0, sales_acct_id=252629163.0, operation_type=u'INSE

In [27]:
contest_sol_end_customer.count()

12197836

In [32]:
contest_sol_end_customer.where(F.col('province').isNull()).count()

12197836

In [34]:

grouped = contest_sol_end_customer.map(lambda x : (x.party_id / 100, x)).groupByKey().mapValues(lambda x : [row.asDict().values() for row in x]).cache()


In [35]:
grouped.toDF(['key', 'list'], sampleRatio = 1).select(['list']).write.saveAsTable("ignite.contest_savm_compressed", mode = 'overwrite')